In [ ]:
import numpy as np
import pandas as pd
import pickle
import matplotlib.pyplot as plt 

In [15]:
import torch
import torch.utils.data

In [2]:
li_currency = ["USDJPY", "EURUSD", "GBPUSD", "USDCHF",
               "USDCAD", "TRYUSD", "AUDUSD", "NZDUSD", ]
DATA_FILE_NAME = "../datas/model/data_hour.csv"


In [3]:
df = pd.read_csv(DATA_FILE_NAME)
df.tail()


,USDJPY_0,USDJPY_1,USDJPY_2,USDJPY_3,USDJPY_4,USDJPY_5,USDJPY_6,USDJPY_7,USDJPY_8,USDJPY_9,...,AUDUSD_price_target,NZDUSD_price_target,USDJPY_price_diff_target,EURUSD_price_diff_target,GBPUSD_price_diff_target,USDCHF_price_diff_target,USDCAD_price_diff_target,TRYUSD_price_diff_target,AUDUSD_price_diff_target,NZDUSD_price_diff_target
30746,1,1,2,1,1,1,1,2,1,1,...,0.00035,0.00060,-0.065,-0.00040,-0.00065,0.00125,0.00175,-0.0035,-0.00025,-0.00090
30747,1,2,1,1,1,1,2,1,1,1,...,0.00005,-0.00105,-0.140,0.00010,-0.00190,0.00155,0.00175,-0.0130,-0.00065,-0.00365
30748,1,2,1,1,1,1,2,1,2,1,...,-0.00025,-0.00055,-0.105,0.00065,-0.00200,-0.00045,0.00060,-0.0340,-0.00105,-0.00265
30749,1,1,2,1,1,1,1,1,1,2,...,-0.00020,-0.00055,-0.090,0.00105,-0.00135,-0.00050,0.00005,-0.0285,-0.00065,-0.00210
30750,1,1,1,2,1,1,1,1,2,2,...,-0.00025,-0.00100,0.025,0.00035,-0.00100,0.00015,-0.00025,-0.0285,-0.00065,-0.00220


In [4]:
def columns():
    columns = []
    for i in li_currency:
        column_name = [i + "_" + str(j) for j in range(45)]
        columns += (column_name[:42] + [i + "_price"])
    columns += [i + "_diff" for i in columns]
    return columns

def columns2():
    columns = []
    for i in li_currency:
        column_name = [i + "_" + str(j) for j in range(45)]
        columns += (column_name[:42] + [i + "_price"])
    columns = [i + "_diff" for i in columns]
    return columns


def price_columns():
    columns = []
    for i in li_currency:
        columns += ([i + "_price"])
    columns += [i + "_diff" for i in columns]
    return columns


def target_columns():
    columns = []
    for i in li_currency:
        columns += ([i + "_price_target"])
    return columns


In [5]:
df_target = df[target_columns()]
df_target.tail()

,USDJPY_price_target,EURUSD_price_target,GBPUSD_price_target,USDCHF_price_target,USDCAD_price_target,TRYUSD_price_target,AUDUSD_price_target,NZDUSD_price_target
30746,-0.085,-0.00030,0.00005,0.00025,0.00040,0.0005,0.00035,0.00060
30747,-0.115,-0.00005,-0.00055,0.00035,0.00015,-0.0110,0.00005,-0.00105
30748,-0.090,0.00005,-0.00090,-0.00025,0.00000,-0.0235,-0.00025,-0.00055
30749,-0.130,0.00030,-0.00105,-0.00060,0.00000,-0.0135,-0.00020,-0.00055
30750,-0.060,-0.00015,-0.00110,-0.00025,-0.00010,-0.0130,-0.00025,-0.00100


In [41]:
datas = df[columns()]
datas.tail()

,USDJPY_0,USDJPY_1,USDJPY_2,USDJPY_3,USDJPY_4,USDJPY_5,USDJPY_6,USDJPY_7,USDJPY_8,USDJPY_9,...,NZDUSD_33_diff,NZDUSD_34_diff,NZDUSD_35_diff,NZDUSD_36_diff,NZDUSD_37_diff,NZDUSD_38_diff,NZDUSD_39_diff,NZDUSD_40_diff,NZDUSD_41_diff,NZDUSD_price_diff
30746,1,1,2,1,1,1,1,2,1,1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00150
30747,1,2,1,1,1,1,2,1,1,1,...,0.0,-1.0,1.0,-1.0,1.0,0.0,0.0,0.0,-1.0,-0.00260
30748,1,2,1,1,1,1,2,1,2,1,...,0.0,-1.0,1.0,-1.0,1.0,0.0,0.0,0.0,-1.0,-0.00210
30749,1,1,2,1,1,1,1,1,1,2,...,0.0,-1.0,1.0,-1.0,1.0,0.0,0.0,0.0,-1.0,-0.00155
30750,1,1,1,2,1,1,1,1,2,2,...,0.0,-1.0,1.0,-1.0,1.0,0.0,0.0,0.0,-1.0,-0.00120


In [42]:
def classify(x, y=1000):
    x *= y
    if x > 80:
        return 2
    elif x > 40:
        return 1
    elif x > -40:
        return 0
    elif x > -80:
        return -1
    else:
        return -2


In [43]:
from sklearn.preprocessing import StandardScaler

In [44]:
targets = df_target[[i+"_price_target" for i in li_currency]]
sc = StandardScaler()
targets = sc.fit_transform(targets)

In [ ]:
plt.hist(targets[:,0],bins=100)

In [45]:
targets.shape, train_target.shape, test_target.shape

((30751, 8), torch.Size([30651, 8]), torch.Size([100, 8]))

In [46]:
train_data = datas[:-100]
test_data = datas[-100:]

train_target = targets[:-100]
test_target = targets[-100:]

In [47]:
train_data = torch.Tensor(train_data.values)
test_data = torch.Tensor(test_data.values)

train_target = torch.Tensor(train_target)
test_target = torch.Tensor(test_target)

In [48]:
train_dataset = torch.utils.data.TensorDataset(train_data,train_target)
test_dataset = torch.utils.data.TensorDataset(test_data,test_target)

In [49]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True)

In [50]:
import torch.nn as nn
import torch.nn.functional as F

class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(688, 128)
        self.fc2 = nn.Linear(128, 128)
        self.fc3 = nn.Linear(128, 128)
        self.fc4 = nn.Linear(128, 8)
        
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

In [51]:
model = Model()
model.train()

Model(
  (fc1): Linear(in_features=688, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=8, bias=True)
)

In [52]:
import torch.optim as optim

In [53]:
epochs = 100
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
losses = []
for epoch in range(epochs):
    total_loss = 0.0
    for data in train_loader:
        inputs, labels = data
        # labels = torch.from_numpy(np.asarray(labels))
        # inputs, labels, = inputs.to(device), labels.to(device)
        optimizer.zero_grad()  
        outputs = model(inputs)
        loss_function = nn.MSELoss()
        loss = loss_function(outputs, labels.view_as(outputs))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(total_loss)
    losses.append(total_loss)

959.5973215848207
957.7036306262016
956.8588425219059
955.6519608199596
953.4283434152603
951.8876187801361
948.165974766016
945.620475590229
942.4092110693455
938.2457921504974
936.2608716189861
932.0605062246323
927.0104047060013
926.313050121069
920.0086445808411
916.6935924142599
910.3529517650604
911.4396142214537
911.1999161690474
907.9812406003475
902.231707945466
893.8637572228909
890.3495133817196
887.9400936365128
879.5906287133694
876.4345248043537
879.6552082598209
863.1064534336329
862.694701731205
858.8243922293186
856.0261997282505
853.7807831168175
851.0942438691854
840.6899072229862
840.7028614580631
827.3784765303135
818.2213476002216
820.896717607975
809.6244122982025
800.161920696497
797.5565362423658
791.2529729604721
801.0380403399467
789.0320988297462
778.1395652890205
779.5895756185055
768.1696796417236
765.8407345414162
754.7085953652859
751.672853320837
737.1989223659039
732.0359064042568
722.3279795199633
721.1158154457808
710.8838479965925
703.4357422143221


In [54]:
model.eval()

Model(
  (fc1): Linear(in_features=688, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=8, bias=True)
)

In [ ]:
actual_labels, pred_labels = np.empty(0).reshape(0,8), np.empty(0).reshape(0,8)
score = 0
for inputs, labels in test_loader:
    # labels = torch.from_numpy(np.asarray(labels))
    # inputs, labels = inputs.to(device), labels.to(device)
    outputs = model(inputs)
    actual_labels = np.concatenate([actual_labels, np.array(labels.cpu().numpy())])
    pred_labels = np.concatenate([pred_labels, np.array(outputs.cpu().detach().numpy())])
    score += loss

In [29]:
actual_labels.shape, pred_labels.shape

((100, 8), (100, 8))

In [30]:
from sklearn.metrics import mean_squared_error
from scipy.stats import pearsonr

In [ ]:
for i in range(8):
    al = actual_labels[:,i]
    pl = pred_labels[:,i]
    print(pearsonr(al, pl)[0])
    plt.scatter(al,pl)
    plt.title(li_currency[i])
    plt.xlabel("actual_labels")
    plt.ylabel("pred_labels")
    plt.show()

In [55]:
filename = "./model.sav"
torch.save(model.to('cpu').state_dict(), './model.pth')

In [40]:
model_new = Model()
model_new.load_state_dict(torch.load('model.pth'))
model_new.eval()

Model(
  (fc1): Linear(in_features=344, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=128, bias=True)
  (fc3): Linear(in_features=128, out_features=128, bias=True)
  (fc4): Linear(in_features=128, out_features=8, bias=True)
)